<div style = "background-color:indigo"><center>
<h1 style="font-size: 50px; font-weight: bold; color:goldenrod; border-top: 3px solid goldenrod; padding-top: 10px">AI California Legislative Policy Analysis (CALPA)</h1>
<div style="font-size: 35px; font-weight: bold; color: goldenrod"> Scratch Notebook</div>
<div style="font-size: 30px; font-weight: bold; color: goldenrod; border-bottom: 3px solid goldenrod; padding-bottom: 20px">v.1.0 April 2025</div>
</center></div>

This is a scratch notebook used to test the code and functionality of the AI California Legislative Policy Analysis (CALPA) system. It is not intended for production use and may contain incomplete or experimental code. The purpose of this notebook is to facilitate the development and testing of the CALPA system, including its data processing, analysis, and visualization components. The notebook may include code snippets, comments, and notes related to the development process. Please refer to the official documentation and user guides for the CALPA system for more information on its usage and features.

In [ ]:
# %reset

<h1 style="font-weight:bold; color:orangered; border-bottom: 2px solid orangered">Basic Code Segment</h1>

In [1]:
# Import required libraries
import os, time
from datetime import date, datetime
import json, mimetypes, glob, base64, zipfile, io, dotenv
import requests
import pandas as pd

In [8]:
# Load the Calpa module located in the scripts/python/calpa directory
from calpa.calpa import Calpa
from calpa.legiscan import LegiScan

# Load environment variables from .env file
dotenv.load_dotenv(os.path.join(os.getcwd(), ".env"))

# Instantiate the LegiScan and Calpa classes
calpa = Calpa()
legiscan = LegiScan()

# Create project metadata for the AI project
prjMetadata = calpa.projectMetadata("AI", "0")

# Create the project directories dictionary
prjDirs = calpa.projectDirectories(os.getcwd())

Project Global Settings:
- Name: California Legislative Policy Analysis
- Title: AI Legislative Policy Analysis
- Version: 1.0
- Author: Dr. Kostas Alexandridis, GISP
Data Dates
- Start Date: 2010-12-02
- End Date: 2025-04-18
- Periods: 2009-2010, 2011-2012, 2013-2014, 2015-2016, 2017-2018, 2019-2020, 2021-2022, 2023-2024, 2025-2026
Directory Global Settings:

General:
- Project: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA
- Admin: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\admin
- Metadata: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\metadata
- Analysis: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\analysis
Scripts:
- Python Calpa Module: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\calpa
- Markdown Scripts: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\markdown
- RIS Scripts: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\ris
Data:
- Main Data: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\data
- Documents: c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\data\docs
- Le

In [9]:
# Get the list of sessions from LegiScan
sessionList = legiscan.getSessionList()

# Convert the sessionList to a pandas DataFrame
sessionDf = pd.DataFrame(sessionList)
sessionDf.head()

,2025-2026,2023-2024,2021-2022,2019-2020,2017-2018,2015-2016,2013-2014,2011-2012,2009-2010
session_id,2172,2016,1791,1624,1400,1120,993,82,30
state_id,5,5,5,5,5,5,5,5,5
state_abbr,CA,CA,CA,CA,CA,CA,CA,CA,CA
year_start,2025,2023,2021,2019,2017,2015,2013,2011,2009
year_end,2026,2024,2022,2020,2018,2016,2014,2012,2010


In [ ]:
aiBills = legiscan.getStoredBills("AI")
lcBills = legiscan.getStoredBills("LC")

In [ ]:
# Load the sessionList from the json file in the metadata directory
sessionListJson = os.path.join(prjDirs["pathMetadata"], "sessionList.json")
with open(sessionListJson, "r", encoding="utf-8") as f:
    sessionListStored = json.load(f)

# Compare the sessionList and sessionListStored dictionaries for any changes
unmatchedSessions = legiscan.matchHash(
    sessionList, sessionListStored, "session_hash", silent=True
)

# if the unmatchedSessions is empty, print "All sessions match", and delete the unmatchedSessions variable
if unmatchedSessions is None:
    print("All sessions match")
    del unmatchedSessions

In [ ]:
# Get the list of session people from LegiScan
sessionPeople = {}
for key, value in sessionList.items():
    sessionId = value["session_id"]
    sessionPeople[key] = legiscan.getSessionPeople(sessionId)

# Load the sessionPeople from the json file in the metadata directory
sessionPeopleJson = os.path.join(prjDirs["pathMetadata"], "sessionPeople.json")
with open(sessionPeopleJson, "r", encoding="utf-8") as f:
    sessionPeopleStored = json.load(f)

# Compare the sessionPeople and sessionPeopleStored dictionaries for any changes
unmatchedPeople: dict[str, dict[str, dict]] = {}
for key, value in sessionPeople.items():
    unmatchedPeople[key] = {}
    for personKey, personValue in value["people"].items():
        unmatchedPeople[key] = legiscan.matchHash(
            sessionPeople[key]["people"],
            sessionPeopleStored[key]["people"],
            "person_hash",
            silent=True,
        )

# if all sections of the unmatched dictionary are empty, print "all empty"
if all(not value for value in unmatchedPeople.values()):
    print("All sections are matched.")
    # Delete the unmatched dictionary
    del unmatchedPeople

In [ ]:
legiscan.updateStoredBills("AI", "XX999", 123555)

In [ ]:
legiscan.getStoredBills("AI")

In [ ]:
# Read the aiBills.json file from the data/lookup directory
aiBillsJson = os.path.join(prjDirs["pathDataLookup"], "aiBills.json")
with open(aiBillsJson, "r", encoding="utf-8") as f:
    aiBills = json.load(f)

In [ ]:
# Write the aiBills dictionay to a pandas DataFrame
aiBillsDf = pd.DataFrame(aiBills)
aiBillsDf.head()